# INGESTÃO DE DADOS
---
É preciso agora ler o arquivo contendo as atribuições dos perfis do SAP e enriquecê-lo com os dados extraídos da base integrada de RH.

In [1]:
import os
import duckdb
import pandas as pd
import numpy as np
import random
import json

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [2]:
BIRH_DATA              = os.environ['BIRH_DATA']
SAP_USER_ROLES_CSV     = os.environ['SAP_USER_ROLES_CSV']
DATASET                = os.environ['DATASET']
PROB_ROLE_REMOVAL      = float(os.environ['PROB_ROLE_REMOVAL'])
REMOVED_ROLES          = os.environ['REMOVED_ROLES']
USER_ROLES             = os.environ['USER_ROLES']
ORGUNIT_ROLES          = os.environ['ORGUNIT_ROLES']
FUNCTION_ROLES         = os.environ['FUNCTION_ROLES']
SKIP_ROLES             = set(json.loads(os.environ["SKIP_ROLES"]))

print(f"""
BIRH_DATA              = {BIRH_DATA}
SAP_USER_ROLES_CSV     = {SAP_USER_ROLES_CSV}
DATASET                = {DATASET}
PROB_ROLE_REMOVAL      = {PROB_ROLE_REMOVAL}
REMOVED_ROLES          = {REMOVED_ROLES}
USER_ROLES             = {USER_ROLES}
ORGUNIT_ROLES          = {ORGUNIT_ROLES}
FUNCTION_ROLES         = {FUNCTION_ROLES}
SKIP_ROLES             = {SKIP_ROLES}
""")


BIRH_DATA              = ./DATA/birh.parquet
SAP_USER_ROLES_CSV     = ./DATA/chaveXperfil.csv
DATASET                = ./DATA/EXPERIMENTO-01-B/dataset.parquet
PROB_ROLE_REMOVAL      = 0.1
REMOVED_ROLES          = ./DATA/EXPERIMENTO-01-B/removed-roles.parquet
USER_ROLES             = ./DATA/EXPERIMENTO-01-B/user-roles.parquet
ORGUNIT_ROLES          = ./DATA/EXPERIMENTO-01-B/orgunit-roles.parquet
FUNCTION_ROLES         = ./DATA/EXPERIMENTO-01-B/function-roles.parquet
SKIP_ROLES             = {'Z:FI_AA_PB001_EXE_CON_REL_IMOB', 'Z:BC_USO_GERAL'}



Os dados do BIRH extraídos anteriormente são lidos

In [3]:
# carrega dados do BIRH
birh_df = pd.read_parquet(BIRH_DATA)
birh_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91830 entries, 0 to 91829
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   chave_usuario            91798 non-null  object
 1   tipo_usuario             91830 non-null  object
 2   centro_custo             39913 non-null  object
 3   lotacao_topo             91830 non-null  object
 4   sigla_lotacao            91830 non-null  object
 5   nome_lotacao             91830 non-null  object
 6   cargo                    91830 non-null  object
 7   enfase                   91830 non-null  object
 8   funcao                   91830 non-null  object
 9   sindicato                39415 non-null  object
 10  area_rh                  91296 non-null  object
 11  imovel                   91296 non-null  object
 12  local_negocio            91293 non-null  object
 13  grupo_prestacao_servico  51150 non-null  object
 14  regime_trabalho          76661 non-nul

As atribuições de perfis do SAP estão presente em um arquivo CSV delimitado por ';' contendo duas colunas, o usuário e o perfil atribuído. Este arquivo é gerado por pessoas da área de Segurança da Informação e disponibilizado para processamento.

In [5]:
# carrega arquivo de perfis de usuários do SAP
sap_user_roles_df = pd.read_csv(SAP_USER_ROLES_CSV, delimiter=';')
sap_user_roles_df.rename(columns={'UNAME': 'chave_usuario', 'AGR_NAME': 'role'}, inplace=True)
sap_user_roles_df = sap_user_roles_df[~sap_user_roles_df['role'].isin(SKIP_ROLES)].copy()
sap_user_roles_df

,role,chave_usuario
11466,Z:MM_PB0020_CONFIG_MODELO_LV,BYK8
11467,Z:MM_PB0020_CONFIG_MODELO_LV,CKJI
11468,Z:MM_PB0020_CONFIG_MODELO_LV,JG55
11469,Z:MM_PB0020_CONFIG_MODELO_LV,JGEJ
11470,Z:MM_PB002_ADM_DPT_EXT_MAT,CZWG
...,...,...
593853,Z:BC_PB001_USUARIO_EXEC_SCRIPT,BI3G
593854,Z:BC_PB001_USUARIO_EXEC_SCRIPT,BIAM
593855,Z:BC_PB001_USUARIO_EXEC_SCRIPT,BIAM
593856,Z:BC_PB001_USUARIO_EXEC_SCRIPT,BIGW


A biblioteca **DuckDB**, um sistema gerenciador de bancos de dados baseado em SQL e executado no mesmo processo da aplicação, é utilizado diversas vezes no decorrer desse trabalho para processar dados em memória (DataFrames pandas), em disco (arquivos parquet) e dados mistos(parte em memória e parte em disco numa mesma query) de maneira transparente.

A consulta abaixo realiza a junção dos dados do BIRH com as atribuições dos perfis do SAP.

Inicialmente esta etapa era realizada através da construção de um PIVOT em cima da tabela de perfis, transformando cada perfil em uma coluna cujo valor indicava a atribuição ou não atribuição do perfil para o usuário vigente.

Esta estratégia apresenta uma série de problemas relacionados com consumo excessivo de memória, pois estamos trabalhando com aproximadamente 4 mil perfis esparsamente atribuídos para os 85.000+ usuários.

Após alguma pesquisa, foi possível constatar que armazenar os nomes dos perfis em um atributo multivalorado chamado *roles* reduzia o consumo de memória em uma ordem de magnitude. 

Segue abaixo a query utilizada:

In [ ]:
conn = duckdb.connect(':memory:')

In [ ]:
sql = """
    WITH cte_user_roles AS (
        SELECT  a.chave_usuario
        ,       LIST(DISTINCT a.role) AS roles
        FROM    sap_user_roles_df AS a
        GROUP   BY a.chave_usuario    
    )
    SELECT  a.chave_usuario
    ,       a.tipo_usuario
    ,       a.centro_custo
    ,       a.lotacao_topo
    ,       a.sigla_lotacao
    ,       a.nome_lotacao
    ,       a.cargo
    ,       a.enfase
    ,       a.funcao
    ,       a.sindicato
    ,       a.area_rh
    ,       a.imovel
    ,       a.local_negocio
    ,       a.grupo_prestacao_servico
    ,       a.regime_trabalho
    ,       a.empresa_contrato
    ,       a.tipo_localizacao
    ,       a.status_cracha
    ,       a.situacao_cracha
    ,       a.objetos_contratos
    ,       a.cursos    
    ,       COALESCE(b.roles, []) AS roles        
    FROM    birh_df a
            --
            LEFT OUTER JOIN cte_user_roles b
            ON  a.chave_usuario          = b.chave_usuario
            --
    WHERE   a.chave_usuario              IS NOT NULL
    ORDER   BY a.chave_usuario
"""
dataset_df = conn.execute(sql).fetchdf()
dataset_df.to_parquet(DATASET)
dataset_df.info(verbose=True, show_counts=True)

Para testar a acurácia do recomendador iremos aleatoriamente escolher funcionários com a probabilidade **PROB_PROFILE_REMOVAL** e remover, um de seus perfis. Posteriormente será avaliado se o perfil recomendado consta entre as *TOP 1, TOP 3 e TOP 5* recomendações. Estes dados serão agregados e utilizados posteriormente para testar o efeito das escolhas dos hiperparâmetros nas recomendações 

In [ ]:
def remove_random_role(roles):
    if not roles:
        return [], None
    count = len(roles)
    remove_idx = random.choice(range(count))
    removed = roles.pop(remove_idx)
    return roles, removed


roles = dataset_df.iloc[60008]["roles"]
remove_random_role(roles)
rows, _ = dataset_df.shape
chaves = dataset_df["chave_usuario"]
roles = dataset_df["roles"]
probs = [np.random.rand() for i in range(rows)]
removed_chaves = []
removed_roles = []
new_roles = []

for chave, candidate_roles, prob in zip(chaves, roles, probs):
    if prob > PROB_ROLE_REMOVAL:
        new_roles.append(candidate_roles)
        continue
    roles_updated, removed_role = remove_random_role(candidate_roles)
    
    if removed_role is None:
        new_roles.append([])
        continue
    
    new_roles.append(roles_updated)
    removed_chaves.append(chave)
    removed_roles.append(removed_role)
    
# cria a listagem de perfis removidos
removed_roles_df = pd.DataFrame({
    "chave_usuario": removed_chaves
,   "role": removed_roles
})
removed_roles_df.to_parquet(REMOVED_ROLES)

# atualiza conjunto de dados com o perfis removidos
dataset_df["roles"] = new_roles
dataset_df.to_parquet("DATASET")

# exibe data frames
display(removed_roles_df.info(verbose=True, show_counts=True))
display(dataset_df.info(verbose=True, show_counts=True))

Gerado o conjunto de dados principal, gera-se uma listagem chave de usuário x perfil em formato parquet para consultar posteriormente:

1 - Quais os perfis que os usuários vizinhos de uma pessoa tem; e

2 - Quais perfis uma pessoa já tem para que os mesmos não sejam indicados como perfis candidatos pelo recomendador de perfis

In [ ]:
sql = """
WITH cte_dados AS (
    SELECT  DISTINCT 
            a.chave_usuario
    ,       UNNEST(a.roles) AS role
    FROM    dataset_df a
)
SELECT  a.chave_usuario
,       a.role
FROM    cte_dados a
WHERE   a.chave_usuario          <> 'n/a'
AND     a.role                   IS NOT NULL
ORDER   BY a.chave_usuario
,       role
"""
user_roles_df = conn.execute(sql).fetchdf()
user_roles_df.to_parquet(USER_ROLES)
user_roles_df.info(verbose=True, show_counts=True)

É preciso verificar que perfis estão atribuídos a cada lotação da empresa pois quando calcula-se a vizinhança de um usuário, os limites organizacionais não irão ser respeitados, principalmente em casos onde a sub-árvore de unidades organizacionais for parcamente povoada.

O recomendador sendo desenvolvido não irá sugerir perfis que não possuam ao menos 3 atribuições a uma lotação topo, visando eliminar recomendações de perfis que, apesar de figurarem na vizinhança de um usuário, podem não ser relevantes para recomendação.

De maneira análoga em um recomendador de filmes, seria o equivalente de recomendar um filme com censura 18+ para um adolescente de 15 anos.

In [ ]:
sql = """
WITH cte_dados AS (
    SELECT  a.lotacao_topo
    ,       a.sigla_lotacao
    ,       UNNEST(a.roles) AS role
    FROM    dataset_df a
)
SELECT  a.lotacao_topo
,       a.sigla_lotacao
,       a.role
,       COUNT(*) as atribuicoes
FROM    cte_dados a
WHERE   a.sigla_lotacao          <> 'n/a'
AND     a.role                   IS NOT NULL
GROUP   BY a.sigla_lotacao
,       a.lotacao_topo
,       a.role
HAVING  COUNT(*) >= 3
ORDER   BY a.sigla_lotacao
,       role
"""
orgunit_roles_df = conn.execute(sql).fetchdf()
orgunit_roles_df.to_parquet(ORGUNIT_ROLES)
orgunit_roles_df.info(verbose=True, show_counts=True)

Ao gerar uma recomendação, o fato do perfil possuir atribuições pré-existentes para usuários com o mesmo tipo de usuário, cargo, ênfase ou função gratificada deverá ser levado em conta para determinar a qualidade da recomendação.

Por existirem diferenças significativas entre os perfis atribuídos para funcionários e prestadores de serviço, o recomendador de perfis somente recomenda perfis que já possuam atribuições para o mesmo tipo de funcionário.

In [ ]:
sql = """
WITH cte_dados AS (
    SELECT  a.tipo_usuario
    ,       a.cargo
    ,       a.enfase
    ,       a.funcao
    ,       UNNEST(a.roles) AS role
    FROM    dataset_df a
)
SELECT  a.tipo_usuario
,       a.cargo
,       a.enfase
,       a.funcao
,       a.role
,       COUNT(*) as atribuicoes
FROM    cte_dados a
WHERE   1=1
AND     a.role                   IS NOT NULL
GROUP   BY a.tipo_usuario
,       a.cargo
,       a.enfase
,       a.funcao
,       a.role
HAVING  COUNT(*) >= 3 -- número mínimo de ocorrências
ORDER   BY a.tipo_usuario
,       a.cargo
,       a.enfase
,       a.funcao
,       a.role
"""
function_roles_df = conn.execute(sql).fetchdf()
function_roles_df.to_parquet(FUNCTION_ROLES)
function_roles_df.info(verbose=True, show_counts=True)

Dado um usuário para recomendação de perfis, a existência de atribuições prévias para:

 - a mesma lotação topo;
 - a mesma lotação;
 - o mesmo tipo de usuário;
 - o mesmo cargo (ou falta de, no caso de prestadores de serviço);
 - a mesma ênfase (ou falta de, no caso de prestadores de serviço); e
 - a mesma função gratificada (ou falta de, no caso de prestadores de serviço e funcionários)
 
 São conjuntamente chamadas de atribuições categóricas e elas figuram de maneira proeminente no mecanismo de scoring das recomendações desenvolvido
 